# Advanced Tutorial

In the basic tutorial we covered how to add static features, predictors and outcomes.
In this tutorial, we'll expand on that, covering how to effectively add many features by:
1. Creating feature combinations from specifications,
2. Using caching, so you can iterate on your datasets without having to complete full computations every time


## Creating feature combinations
Manually specifying a handful of features one at a time is rather straightforward, but what if you want to generate hundreds of features? Or want to have multiple different lookbehind windows, e.g. a month, 6 months and a year? Then the amount of code you'll have to write will grow quite substantially and becomes time consuming and hard to navigate.

To solve this problem, we implemented feature group specifications. They allow you to combinatorially create features. Let's look at an example:


In [9]:
from pprint import pprint as pprint

import numpy as np
from timeseriesflattener.aggregation_fns import maximum, mean
from timeseriesflattener.feature_specs.group_specs import NamedDataframe, PredictorGroupSpec
from timeseriesflattener.testing.load_synth_data import load_synth_predictor_float

In [10]:
pred_spec_batch = PredictorGroupSpec(
    named_dataframes=[
        NamedDataframe(df=load_synth_predictor_float(), name="synth_predictor_float")
    ],
    lookbehind_days=[(0, 365), (365, 730), 1095],
    fallback=[np.nan],
    aggregation_fns=[mean, maximum],
).create_combinations()

You'll note that:

1. All attributes are now required to be lists. This makes iteration easier when creating the combinations.
2. We require a named_dataframes sequence. A namedataframe is exactly that; a dataframe and a name. This is used when we create the features in the output, e.g. for a predictor, the output feature using load_synth_predictor_flaot will be called pred_synth_predictor_float_<metadata> because that's the name attributed in the NamedDataframe.

Let's check that the results look good.

In [11]:
# Create a small summary to highlight the generated predictors
pred_spec_batch_summary = [
    {
        "feature_name": pred_spec.feature_base_name,
        "lookbehind_days": pred_spec.lookbehind_period,
        "aggregation_fn": pred_spec.aggregation_fn.__name__,
    }
    for pred_spec in pred_spec_batch
]
print(f"––––––––– We created {len(pred_spec_batch)} combinations of predictors. ––––––––––")
pprint(pred_spec_batch_summary)

––––––––– We created 6 combinations of predictors. ––––––––––
[{'aggregation_fn': 'mean',
  'feature_name': 'synth_predictor_float',
  'lookbehind_days': LookPeriod(min_days=0.0, max_days=365.0)},
 {'aggregation_fn': 'maximum',
  'feature_name': 'synth_predictor_float',
  'lookbehind_days': LookPeriod(min_days=0.0, max_days=365.0)},
 {'aggregation_fn': 'mean',
  'feature_name': 'synth_predictor_float',
  'lookbehind_days': LookPeriod(min_days=365.0, max_days=730.0)},
 {'aggregation_fn': 'maximum',
  'feature_name': 'synth_predictor_float',
  'lookbehind_days': LookPeriod(min_days=365.0, max_days=730.0)},
 {'aggregation_fn': 'mean',
  'feature_name': 'synth_predictor_float',
  'lookbehind_days': LookPeriod(min_days=0, max_days=1095.0)},
 {'aggregation_fn': 'maximum',
  'feature_name': 'synth_predictor_float',
  'lookbehind_days': LookPeriod(min_days=0, max_days=1095.0)}]


Now we know how to create a bunch of feature specifications quickly! But with more features comes more computation. Let's look at caching next, so we can iterate on our datasets more quickly.

## Caching

Timeseriesflattener ships with a class that allows for caching to disk. Let's look at an example of that:

In [12]:
from pathlib import Path

from skimpy import skim
from timeseriesflattener.feature_cache.cache_to_disk import DiskCache
from timeseriesflattener.flattened_dataset import TimeseriesFlattener
from timeseriesflattener.testing.load_synth_data import load_synth_prediction_times

In [13]:
ts_flattener = TimeseriesFlattener(
    prediction_times_df=load_synth_prediction_times(),
    entity_id_col_name="entity_id",
    timestamp_col_name="timestamp",
    n_workers=4,
    cache=DiskCache(feature_cache_dir=Path(".tmp") / "feature_cache"),
    drop_pred_times_with_insufficient_look_distance=True,
)

2024-01-18 11:38:02 [INFO] Overriding pred_time_uuid_col_name in cache with pred_time_uuid_col_name passed to init of flattened dataset


All we need to specify is that we use the DiskCache class, and which directory to save the feature cache to.

The first time we create features, this will just save them to disk and won't make any difference to performance. But say we want to add two more features - then it'll load the features that it has already computed from disk, and then only compute the two new features.

Note that DiskCache is an instance of the abstract class FeatureCache. If you want to implement your own cache, for example using REDIS or SQL, all you'll need is to implement the 3 methods in that class. Now, let's compute a dataframe to check that everything works.

In [14]:
ts_flattener.add_spec(pred_spec_batch)

In [15]:
df = ts_flattener.get_df()

2024-01-18 11:38:03 [INFO] There were unprocessed specs, computing...
2024-01-18 11:38:03 [INFO] _drop_pred_time_if_insufficient_look_distance: Dropped 6053 (60.53%) rows
2024-01-18 11:38:03 [INFO] Processing 6 temporal features in parallel with 4 workers. Chunksize is 2. If this is above 1, it may take some time for the progress bar to move, as processing is batched. However, this makes for much faster total performance.
  0%|          | 0/6 [00:00<?, ?it/s]/Users/au554730/Desktop/Projects/timeseriesflattener/.venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'allow_mutation' has been removed
  warnings.warn(message, UserWarning)
/Users/au554730/Desktop/Projects/timeseriesflattener/.venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'allow_mutation' has been removed
  warnings.warn(message, UserWarning)
/Users/au554730/Desktop/Projects/timeseri

In [16]:
skim(df)

list(df.columns)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 3947   │ │ float64     │ 6     │                                                          │
│ │ Number of columns │ 9      │ │ int64       │ 1     │                                                          │
│ └───────────────────┴────────┘ │ datetime64  │ 1     │                                                          │
│                                │ string      │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name                ┃ NA    ┃ NA %   ┃ mean   ┃ sd     ┃ p0       ┃ p25   ┃ p75   ┃ p100   ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━┩  │
│ │ entity_id                  │     0 │      0 │   5000 │   2900 │        0 │  2600 │  7400 │  10000 │ █████▇ │  │
│ │ pred_synth_predictor       │     7 │   0.18 │      5 │    1.3 │     0.29 │   4.1 │   5.8 │    9.9 │  ▂█▇▁  │  │
│ │ pred_synth_predictor       │   510 │     13 │    6.6 │    2.6 │    0.024 │   4.8 │   8.8 │     10 │ ▂▂▃▄▆█ │  │
│ │ pred_synth_predictor       │   530 │     14 │    6.6 │    2.6 │   0.0084 │   4.8 │   8.8 │     10 │ ▁▂▃▄▆█ │  │
│ │ pred_synth_predictor       │     7 │   0.18 │    8.4 │    1.5 │     0.29 │   7.8 │   9.5 │     10 │    ▁▃█ │  │
│ │ pred_synth_predictor       │   510 │     13 │    5.1 │    2.2 │    0.024 │   3.6 │   6.5 │     10 │ ▂▄██▅▂ │  │
│ │ pred_synth_predictor       │   530 │     14 │      5 │    2.1 │   0.0084 │   3.6 │   6.4 │    9.9 │ ▂▄██▄▂ │  │
│ └────────────────────────────┴───────┴────────┴────────┴────────┴──────────┴───────┴───────┴────────┴────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓  │
│ ┃ column_name      ┃ NA   ┃ NA %    ┃ first                      ┃ last                       ┃ frequency    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩  │
│ │ timestamp        │    0 │       0 │    1968-01-02 05:12:00     │    1969-12-31 21:42:00     │ None         │  │
│ └──────────────────┴──────┴─────────┴────────────────────────────┴────────────────────────────┴──────────────┘  │
│                                                     string                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓  │
│ ┃ column_name                           ┃ NA    ┃ NA %      ┃ words per row            ┃ total words         ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩  │
│ │ prediction_time_uuid                  │     0 │         0 │                        1 │                3900 │  │
│ └───────────────────────────────────────┴───────┴───────────┴──────────────────────────┴─────────────────────┘  │
╰────────────────────────────────────────────────────── 

['entity_id',
 'timestamp',
 'prediction_time_uuid',
 'pred_synth_predictor_float_within_0_to_1095_days_mean_fallback_nan',
 'pred_synth_predictor_float_within_365_to_730_days_maximum_fallback_nan',
 'pred_synth_predictor_float_within_0_to_365_days_maximum_fallback_nan',
 'pred_synth_predictor_float_within_0_to_1095_days_maximum_fallback_nan',
 'pred_synth_predictor_float_within_365_to_730_days_mean_fallback_nan',
 'pred_synth_predictor_float_within_0_to_365_days_mean_fallback_nan']

In [17]:
# For displayability, shorten col names
pred_cols = [c for c in df.columns if c.startswith("pred_")]
rename_dict = {c: f"pred_{i+1}" for i, c in enumerate(pred_cols)}
df_renamed = df.rename(rename_dict, axis=1)

# Print a dataframe
base_cols = ["entity_id", "timestamp", "prediction_time_uuid"]
renamed_cols = list(rename_dict.values())

df_renamed[0:10][base_cols + renamed_cols].style.set_table_attributes('style="font-size: 14px"')

,entity_id,timestamp,prediction_time_uuid,pred_1,pred_2,pred_3,pred_4,pred_5,pred_6
0,9903,1968-05-09 21:24:00,9903-1968-05-09-21-24-00,2.864626,2.194319,0.154981,5.931553,1.408655,0.154981
1,4927,1968-06-30 12:13:00,4927-1968-06-30-12-13-00,4.466599,nan,6.730694,8.630901,nan,4.957251
2,3157,1969-10-07 05:01:00,3157-1969-10-07-05-01-00,4.168456,nan,5.243176,5.243176,nan,5.068323
3,9793,1968-12-15 12:59:00,9793-1968-12-15-12-59-00,7.144959,8.293266,9.708976,9.727182,6.230417,8.091755
4,9861,1969-01-22 17:34:00,9861-1969-01-22-17-34-00,3.669635,5.491415,3.130283,6.217161,3.309197,3.130283
5,657,1969-04-14 15:47:00,657-1969-04-14-15-47-00,7.391514,7.903614,nan,7.903614,7.903614,nan
6,7916,1968-12-20 03:38:00,7916-1968-12-20-03-38-00,4.251704,6.084523,4.318586,6.979156,6.084523,3.901992
7,2883,1968-01-28 21:50:00,2883-1968-01-28-21-50-00,4.712403,nan,8.257742,8.257742,nan,8.257742
8,1515,1968-07-18 08:28:00,1515-1968-07-18-08-28-00,3.112700,3.684614,8.654839,8.654839,3.104674,2.907289
9,6754,1968-09-21 01:27:00,6754-1968-09-21-01-27-00,5.082918,3.102132,2.346644,9.657755,2.324913,2.346644
